In [29]:
pip list

Package           Version
----------------- -----------
appnope           0.1.4
asttokens         2.4.1
autograd          1.7.0
comm              0.2.2
contourpy         1.3.0
cvxopt            1.3.2
cycler            0.12.1
debugpy           1.8.5
decorator         5.1.1
executing         2.1.0
fonttools         4.54.1
ipykernel         6.29.5
ipython           8.27.0
jedi              0.19.1
jupyter_client    8.6.3
jupyter_core      5.7.2
kiwisolver        1.4.7
matplotlib        3.7.2
matplotlib-inline 0.1.7
nest-asyncio      1.6.0
numpy             1.25.2
packaging         24.1
pandas            2.1.0
parso             0.8.4
pexpect           4.9.0
pillow            10.4.0
pip               24.1.2
platformdirs      4.3.6
prompt_toolkit    3.0.47
psutil            6.0.0
ptyprocess        0.7.0
pure_eval         0.2.3
Pygments          2.18.0
pyparsing         3.0.9
python-dateutil   2.9.0.post0
pytz              2024.2
pyzmq             26.2.0
scipy             1.14.1
setuptools    

In [20]:
#q1 a)

import numpy as np
from scipy.optimize import minimize
from cvxopt import matrix, solvers

def obj_loss_function(params,X,y,lamb):

    w = params[:-1]  
    w0 = params[-1]  

    logits = X @ w + w0  # Xw + w0
    exponent = -y * logits  
    log_loss = np.sum(np.log(1 + np.exp(exponent))) 

    reg_term = (lamb / 2) * np.sum(w**2)
    
    total_loss = log_loss + reg_term
    return total_loss



def minBinDev(X, y, lamb):
    #want to return w, w0
    n, d = X.shape
    
    initial_params = np.zeros(d + 1)  

    results = minimize(obj_loss_function,initial_params,args = (X,y,lamb), method = 'bfgs')

    w_opt = results.x[:-1]  #  d elements are w
    w0_opt = results.x[-1]  # last element is w0
    
    return w_opt, w0_opt
    

    

#  usage
X = np.random.randn(100, 1) 
y = np.random.choice([-1, 1], size=100) 
lamb = 0.1  # Regularization parameter

# Find the optimized weights and bias
w_opt, w0_opt = minBinDev(X, y, lamb)

print("Optimized weights:", w_opt)
print("Optimized bias:", w0_opt) 

Optimized weights: [0.07802251]
Optimized bias: -0.07861858256519005


In [25]:
#q1b

def minHinge(X, y, lamb, stabilizer=1e-5):
    n,d = X.shape

    #P in (P,q,G,h)
    P = np.zeros((n+d+1,n+d+1))

    #for size of dimensioanlity, create that size identity matrix in the top left
    P[:d, :d] = lamb * np.eye(d)

    #q = d+1 zeroes, n 1's
    q = np.hstack([np.zeros(d+1), np.ones(n)])

    #G matrix: 2n rows for the slack variables for 2 constraints
    G = np.zeros((2*n, d+1+n))

    #first constraint, G11 (could be 0)
    G[:n, :d] = -y[:, None] * X

    #g12 for w0 (could also be 0)
    G[:n,d]= -y

    #g13, Iden matrix (prob good)
    G[:n,d+1:]= -np.eye(n)

    #second contraint 
    G[n:, d+1:] = -np.eye(n)

    #h, constraint for G, so 2n rows to match, 0n and -1n.
    h = np.hstack([-np.ones(n), np.zeros(n)])

    P += stabilizer * np.eye(d+1+n)

    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)

    solution = solvers.qp(P,q,G,h)

    w = np.array(solution['x'][:d])
    w0 = np.array(solution['x'][d])

    return w,w0














In [26]:
#q1c

#yhat = classify(Xtest, w, w0)

def classify(Xtest, w, w0):

    matrix_weight = np.dot(Xtest,w)+w0

    yhat = np.sign(matrix_weight)

    return yhat

    
